# A1-Q2: fp2dec

In [1]:
# Standard imports
import numpy as np
np.seterr(all='ignore');  # allows floating-point exceptions
import matplotlib.pyplot as plt

In [2]:
# Supplied Code

def fpMath(b1, b2, fcn, t, L, U):
    '''
     b = fpMath(b1, b2, fcn, t, L, U)
    
     Performs a binary arithmetic operation.
     
     Inputs:
       b1 and b2 are the input binary strings, from F(2, t, L, U)
       fcn is a function that takes 2 inputs, and returns a value.
       t, L, and U specify the FPNS.
    
     Output:
       b is a binary string for the nearest value in F(2, t, L, U).
    
     For example, 3 - (-1) in binary, is
    
      fpMath('+0.11b2', '-0.01b2', (lambda z1,z2: z1-z2), 2, -5, 5)
    
     would perform subtraction, and return the string '+0.10b3'.
    '''

    x1 = fp2dec(b1)
    x2 = fp2dec(b2)
    
    y = fcn(x1, x2)
    
    b = dec2fp(y, t, L, U)
    
    return b

def dec2fp(x_orig, t, L, U):
    '''
     b = dec2fp(x, t, L, U)
    
     Converts the number x to a binary floating-point representation,
     rounding to t digits, and L and U as the lower and upper bounds on
     the exponent.
    '''

    x = x_orig
    
    # First, handle the exceptions
    if x==0:
        y = '+0.'
        for k in range(t):
            y += '0'
        y += 'b0'
        return y
    elif x==np.inf:
        return 'Inf'
    elif x==-np.inf:
        return '-Inf'
    elif np.isnan(x):
        return 'NaN'
    
    if x<0:
        x = -x
        bx = '-0.'
    else:
        bx = '+0.'
    
    if x>=1:
        myexp = 0
        while x>=1:
            x /= 2.
            myexp += 1
    else:
        myexp = 1
        while x<1:
            x *= 2.
            myexp  -= 1
        x /= 2.
        
    remainder = x - np.floor(x)
    
    # Process the fractional part for t binary digits
    for d in range(t):
        remainder = remainder * 2.
        if remainder>=1:
            bx += '1'
            remainder -= 1
        else:
            bx += '0'
    
    bx += 'b' + str(myexp)

    # Round up if remainder is >= 0.5
    if remainder>=0.5:
        delta = list(bx)
        delta[3:3+t] = '0'*t
        delta[2+t] = '1'
        bx_up = fpMath(bx, ''.join(delta), (lambda z1,z2: z1+z2), t, L, U);
        #print('Rounding '+bx+' up to '+bx_up+' by adding '+''.join(delta))
        bx = bx_up

    y = bx
    
    if myexp>U:
        y = 'overflow'
    elif myexp<L:
        y = '+0.'
        for k in range(t):
            y += '0'
        y += 'b0'
    
    return y

In [3]:
def fp2dec(B):
    '''
     x = fp2dec(B)
    
     Converts the string B to a decimal value (double-precision).
     Examples:
      fp2dec('+0.11000b1')  ->  1.5
      fp2dec('-0.10101b-2') -> -0.1640625
      fp2dec('-0.101b5')    -> -20
    '''
    
    # ==== YOUR CODE HERE ====
    x = 0.
    
    # create variables, divide the string into mantissa and exponent
    b_index = B.find('b')
    exponent_int = int(B[b_index+1:])
    mantissa_str = B[3:b_index]
    mantissa_length = len(mantissa_str)
    floating_point_str = ''
    
    # if exponent < 0, add zeros before mantissa, decimal point locates at 0
    if exponent_int < 0:
        move_dec_left = abs(exponent_int)
        for var in range(move_dec_left):
            floating_point_str = floating_point_str + '0'
        floating_point_str = floating_point_str + mantissa_str
        dec_point_location = 0
    # if exponent = 0, no change to mantissa, decimal point locates at 0
    if exponent_int == 0:
        floating_point_str = mantissa_str
        dec_point_location = 0
    # if exponent > 0, add zeros after mantissa, decimal point locates at exponent
    if exponent_int > 0:
        floating_point_str = mantissa_str
        if mantissa_length < exponent_int:
            for var in range(exponent_int - mantissa_length):
                floating_point_str = floating_point_str + '0'
        dec_point_location = exponent_int
    
    # integer part of the mantissa is all the numbers before decimal point
    # fractional part of the mantissa is all the numbers after decimal point
    integer_part = floating_point_str[:dec_point_location]
    fractional_part = floating_point_str[dec_point_location:]
    if dec_point_location == 0:
        integer_part = '0'
    if dec_point_location == mantissa_length:
        fractional_part = '0'
    integer_part_len = len(integer_part)
    fractional_part_len = len(fractional_part)
    x_integer = 0.
    x_fractional = 0.
    
    # convert integer/fractional part to decimal, and add them to generate x
    for var in range(len(integer_part)):
        exponent = integer_part_len - var - 1
        base = int(integer_part[var])
        x_integer = x_integer + base*2**exponent
    
    for var in range(len(fractional_part)):
        exponent = -var - 1
        base = int(fractional_part[var])
        x_fractional = x_fractional + base*2**exponent
        
    x = x_integer + x_fractional
    
    # positive/negative depending on the B value provided
    if B[0] == '-':
        x = -x
    
    return x

In [4]:
fp2dec('+0.11000b1')

1.5

In [5]:
fp2dec('-0.10101b-2')

-0.1640625

In [6]:
fp2dec('-0.101b5')

-20.0

In [7]:
fp2dec('-0.10101b0')

-0.65625

In [8]:
fp2dec('-0.101b2')

-2.5

In [9]:
fp2dec('-0.101b3')

-5.0

In [10]:
fp2dec('+0.000b3')

0.0